In [1]:
import numpy as np
import pandas as pd

In [4]:
dev = pd.read_csv("dev.csv")
nyt = pd.read_csv("nyt.csv")
train = pd.read_csv("train.csv")

In [9]:
print(dev.describe())
print(dev.shape)

              index          year        length
count  1.287873e+06  1.282468e+06  1.282468e+06
mean   6.439360e+05  1.978668e+03  5.036234e+00
std    3.717771e+05  2.039045e+02  2.130725e+00
min    0.000000e+00  0.000000e+00  2.000000e+00
25%    3.219680e+05  1.996000e+03  4.000000e+00
50%    6.439360e+05  2.004000e+03  4.000000e+00
75%    9.659040e+05  2.011000e+03  6.000000e+00
max    1.287872e+06  2.022000e+03  2.100000e+01
(1287873, 6)


In [10]:
print(nyt.describe())
print(nyt.shape)

              index          year        length
count  26738.000000  26738.000000  26738.000000
mean   13368.500000   2021.103486      5.201025
std     7718.740085      0.304598      2.302468
min        0.000000   2021.000000      3.000000
25%     6684.250000   2021.000000      4.000000
50%    13368.500000   2021.000000      5.000000
75%    20052.750000   2021.000000      6.000000
max    26737.000000   2022.000000     21.000000
(26738, 5)


In [11]:
print(train.describe())
print(train.shape)

              index          year        length
count  5.151492e+06  5.130159e+06  5.130159e+06
mean   2.575746e+06  1.978306e+03  5.034572e+00
std    1.487108e+06  2.057236e+02  2.128290e+00
min    0.000000e+00  0.000000e+00  1.000000e+00
25%    1.287873e+06  1.996000e+03  4.000000e+00
50%    2.575746e+06  2.004000e+03  4.000000e+00
75%    3.863618e+06  2.011000e+03  6.000000e+00
max    5.151491e+06  2.022000e+03  2.100000e+01
(5151492, 6)


In [14]:
combined_train = pd.concat([train, dev], axis=0)
print(combined_train.shape)

(6439365, 6)


In [18]:
print(combined_train.columns)

Index(['index', 'pubid', 'year', 'answer', 'clue', 'length'], dtype='object')


In [17]:
def p_wi(word, length):
    # returns the log of #word / # answers with the same length
    equal_length_dim = combined_train[combined_train["length"]==length].shape[0]
    same_answer_dim = combined_train[combined_train["answer"]==word].shape[0]
    return np.log(np.divide(same_answer_dim, equal_length_dim))


1530791 0


0.0

In [33]:
def split_into_words(clue):
    return [i.strip().lower() for i in clue.split(" ")]

In [39]:
def all_words(same_answer):
    all_words = []
    clues = same_answer["clue"]
    for index, value in clues.items():
        all_words.extend(split_into_words(value))
    return all_words

def p_c_wi(word, clue, all_words):
    # return the sum of the log probability of the words in the clue
    clue_words = split_into_words(clue)

    log_sum = 0
    for word in clue_words:
        if word in all_words:
            log_sum += np.log(np.divide(1, all_words.count(word)))
    
    return log_sum


In [45]:
def make_answer_index(same_length):
    index = {}
    for answer in same_length["answer"].unique():
        index[answer] = same_length[same_length["answer"]==answer]
    return index

def make_all_words_index(index):
    words_index = {}
    for word, subset in index.iterms():
        words_index[word] = all_words(subset)
    return words_index

In [ ]:
length_index = {length:combined_train[combined_train["length"]==length] for length in combined_train["length"].unique()}
answer_index = {answer:combined_train[combined_train["answer"]==answer] for answer in combined_train["answer"].unique()}
# all_words_index = {answer:all_words(answer_index[answer]) for answer in answer_index}

In [46]:
def log_proba(row):
    # return the best 50 words
    # row is coming from the testing set
    length = row["length"]
    clue = row["clue"]
    word_probas = []
    
    same_length = length_index[length]
    equal_length_dim = same_length.shape[0]
    
    for index, value in same_length.iterrows():
        current_word = value["answer"]
        current_clue = value["clue"]
        
        if (index % 100 == 0):
            print(index)
                
        same_answer = index[current_word]
        all_clue_words = all_words_index[current_word]
        
        same_answer_dim = same_answer.shape[0]
        p_wi = np.log(np.divide(same_answer_dim, equal_length_dim))
        
        word_probas.append((current_word, p_wi+p_c_wi(current_word, current_clue, all_clue_words)))
        
    return word_probas.sort(lambda x: x[1])[:50]

log_proba(nyt.iloc[10])

KeyboardInterrupt: 